# 🔋 EV-QA-Framework Demo: ML Anomaly Detection

Интерактивная демонстрация детекции аномалий в телеметрии батареи электромобиля.

**Что покажем:**
- Pydantic валидация данных
- ML-детекция аномалий (Isolation Forest)
- Визуализация нормальных данных vs аномалий
- Severity classification (CRITICAL/WARNING/INFO)

## 📦 Установка зависимостей

In [ ]:
# Если запускаете в Google Colab, раскомментируйте:
# !pip install pydantic scikit-learn pandas numpy matplotlib seaborn

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Настройка стиля графиков
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Библиотеки загружены")

## 1️⃣ Pydantic Валидация Телеметрии

In [ ]:
from pydantic import BaseModel, Field, validator
from typing import Optional

class BatteryTelemetryModel(BaseModel):
    """Строгая модель телеметрии батареи EV"""
    vin: str = Field(..., min_length=17, max_length=17)
    voltage: float = Field(..., ge=0.0, le=1000.0)
    current: float
    temperature: float = Field(..., ge=-50.0, le=150.0)
    soc: float = Field(..., ge=0.0, le=100.0)
    soh: float = Field(..., ge=0.0, le=100.0)
    timestamp: Optional[datetime] = Field(default_factory=datetime.now)
    
    @validator('vin')
    def validate_vin_format(cls, v):
        if not v.isalnum():
            raise ValueError('VIN должен содержать только буквы и цифры')
        return v.upper()

# Тест валидации
valid_data = {
    "vin": "1HGBH41JXMN109186",
    "voltage": 396.5,
    "current": 125.3,
    "temperature": 35.2,
    "soc": 78.5,
    "soh": 96.2
}

telemetry = BatteryTelemetryModel(**valid_data)
print(f"✅ Валидация пройдена: {telemetry.vin}, {telemetry.voltage}V, {telemetry.temperature}°C")

# Попытка невалидных данных
try:
    invalid = BatteryTelemetryModel(**{**valid_data, "voltage": 1500})
except Exception as e:
    print(f"❌ Невалидные данные отклонены: {str(e)[:80]}...")

## 2️⃣ Генерация Синтетических Данных

In [ ]:
# Генерируем нормальные данные телеметрии
np.random.seed(42)
n_samples = 1000

normal_data = pd.DataFrame({
    'voltage': np.random.normal(400, 5, n_samples),      # 400V ± 5V
    'current': np.random.normal(120, 10, n_samples),     # 120A ± 10A
    'temp': np.random.normal(35, 3, n_samples),          # 35°C ± 3°C
    'soc': np.random.normal(80, 10, n_samples)           # 80% ± 10%
})

# Добавляем аномалии (5% данных)
n_anomalies = 50
anomaly_indices = np.random.choice(n_samples, n_anomalies, replace=False)

# Создаем копию для аномалий
data_with_anomalies = normal_data.copy()

# Инжектируем аномалии
data_with_anomalies.loc[anomaly_indices, 'voltage'] = np.random.uniform(450, 600, n_anomalies)  # Высокое напряжение
data_with_anomalies.loc[anomaly_indices, 'temp'] = np.random.uniform(60, 90, n_anomalies)       # Перегрев

print(f"📊 Сгенерировано {n_samples} точек данных")
print(f"🚨 Добавлено {n_anomalies} аномалий ({n_anomalies/n_samples*100:.1f}%)")
print(f"\nПример нормальных данных:")
print(normal_data.head())

## 3️⃣ ML Детекция Аномалий (Isolation Forest)

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

class EVBatteryAnalyzer:
    """ML-анализатор телеметрии батареи"""
    
    def __init__(self, contamination=0.05, n_estimators=200):
        self.model = IsolationForest(
            contamination=contamination,
            n_estimators=n_estimators,
            random_state=42,
            n_jobs=-1
        )
        self.scaler = StandardScaler()
        
    def analyze(self, df):
        """Анализ телеметрии на аномалии"""
        features = ['voltage', 'current', 'temp']
        X = df[features]
        
        # Нормализация
        X_scaled = self.scaler.fit_transform(X)
        
        # Предсказание
        predictions = self.model.fit_predict(X_scaled)
        scores = self.model.score_samples(X_scaled)
        
        # Результаты
        df['is_anomaly'] = predictions == -1
        df['anomaly_score'] = scores
        
        return df

# Обучение и детекция
analyzer = EVBatteryAnalyzer(contamination=0.05, n_estimators=200)
results = analyzer.analyze(data_with_anomalies.copy())

detected_anomalies = results[results['is_anomaly']]
print(f"\n🔍 Детектировано аномалий: {len(detected_anomalies)} из {len(results)}")
print(f"📊 Точность детекции: {len(detected_anomalies)/n_anomalies*100:.1f}% (ожидалось {n_anomalies})")

## 4️⃣ Визуализация: Напряжение vs Температура

In [ ]:
plt.figure(figsize=(14, 7))

# Нормальные точки
normal_points = results[~results['is_anomaly']]
plt.scatter(normal_points['voltage'], normal_points['temp'], 
           c='green', alpha=0.5, s=30, label='Normal', edgecolors='none')

# Аномалии
anomaly_points = results[results['is_anomaly']]
plt.scatter(anomaly_points['voltage'], anomaly_points['temp'], 
           c='red', alpha=0.8, s=100, label='Anomaly', marker='X', edgecolors='darkred', linewidths=1.5)

plt.xlabel('Voltage (V)', fontsize=12, fontweight='bold')
plt.ylabel('Temperature (°C)', fontsize=12, fontweight='bold')
plt.title('🔋 EV Battery Telemetry: Normal vs Anomalies', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

# Добавляем зоны безопасности
plt.axhline(y=60, color='orange', linestyle='--', linewidth=2, alpha=0.7, label='Temp Warning (60°C)')
plt.axvline(x=430, color='orange', linestyle='--', linewidth=2, alpha=0.7, label='Voltage Warning (430V)')

plt.tight_layout()
plt.show()

print("\n📈 График показывает:")
print("  🟢 Зеленые точки = Нормальная работа батареи")
print("  🔴 Красные крестики = Обнаруженные аномалии")
print("  🟠 Пунктирные линии = Пороги безопасности")

## 5️⃣ Распределение Anomaly Scores

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Гистограмма anomaly scores
axes[0].hist(results['anomaly_score'], bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].axvline(x=-0.5, color='orange', linestyle='--', linewidth=2, label='Warning Threshold')
axes[0].axvline(x=-0.8, color='red', linestyle='--', linewidth=2, label='Critical Threshold')
axes[0].set_xlabel('Anomaly Score', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[0].set_title('📊 Distribution of Anomaly Scores', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot по категориям
box_data = [results[~results['is_anomaly']]['anomaly_score'], 
            results[results['is_anomaly']]['anomaly_score']]
axes[1].boxplot(box_data, labels=['Normal', 'Anomaly'], patch_artist=True,
               boxprops=dict(facecolor='lightblue', color='black'),
               medianprops=dict(color='red', linewidth=2))
axes[1].set_ylabel('Anomaly Score', fontsize=12, fontweight='bold')
axes[1].set_title('📦 Anomaly Score: Normal vs Anomaly', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\n📉 Статистика Anomaly Scores:")
print(f"  Normal - Mean: {results[~results['is_anomaly']]['anomaly_score'].mean():.3f}")
print(f"  Anomaly - Mean: {results[results['is_anomaly']]['anomaly_score'].mean():.3f}")

## 6️⃣ Time Series Visualization

In [ ]:
# Берем первые 200 точек для наглядности
sample_data = results.head(200).copy()
sample_data['time'] = range(len(sample_data))

fig, axes = plt.subplots(3, 1, figsize=(16, 10), sharex=True)

# Voltage
axes[0].plot(sample_data['time'], sample_data['voltage'], color='blue', alpha=0.6, linewidth=1.5)
axes[0].scatter(sample_data[sample_data['is_anomaly']]['time'], 
               sample_data[sample_data['is_anomaly']]['voltage'],
               color='red', s=100, marker='X', zorder=5, label='Anomaly')
axes[0].axhline(y=430, color='orange', linestyle='--', alpha=0.7)
axes[0].set_ylabel('Voltage (V)', fontsize=11, fontweight='bold')
axes[0].set_title('⚡ Battery Telemetry Over Time', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Temperature
axes[1].plot(sample_data['time'], sample_data['temp'], color='orangered', alpha=0.6, linewidth=1.5)
axes[1].scatter(sample_data[sample_data['is_anomaly']]['time'], 
               sample_data[sample_data['is_anomaly']]['temp'],
               color='red', s=100, marker='X', zorder=5)
axes[1].axhline(y=60, color='orange', linestyle='--', alpha=0.7)
axes[1].set_ylabel('Temperature (°C)', fontsize=11, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# SOC
axes[2].plot(sample_data['time'], sample_data['soc'], color='green', alpha=0.6, linewidth=1.5)
axes[2].scatter(sample_data[sample_data['is_anomaly']]['time'], 
               sample_data[sample_data['is_anomaly']]['soc'],
               color='red', s=100, marker='X', zorder=5)
axes[2].set_xlabel('Time (samples)', fontsize=11, fontweight='bold')
axes[2].set_ylabel('SOC (%)', fontsize=11, fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n⏱️ Time series показывает детекцию аномалий в реальном времени")

## 7️⃣ Severity Classification

In [ ]:
def assess_severity(score):
    """Оценка серьезности аномалии"""
    if score < -0.8:
        return 'CRITICAL'
    elif score < -0.5:
        return 'WARNING'
    return 'INFO'

results['severity'] = results['anomaly_score'].apply(assess_severity)

# Подсчет по severity
severity_counts = results[results['is_anomaly']]['severity'].value_counts()

# Pie chart
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

colors = {'CRITICAL': 'red', 'WARNING': 'orange', 'INFO': 'yellow'}
axes[0].pie(severity_counts.values, labels=severity_counts.index, autopct='%1.1f%%',
           colors=[colors.get(x, 'gray') for x in severity_counts.index],
           startangle=90, textprops={'fontsize': 12, 'fontweight': 'bold'})
axes[0].set_title('🚨 Anomaly Severity Distribution', fontsize=13, fontweight='bold')

# Bar chart
severity_counts.plot(kind='bar', ax=axes[1], color=[colors.get(x, 'gray') for x in severity_counts.index],
                    edgecolor='black', linewidth=1.5)
axes[1].set_xlabel('Severity Level', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[1].set_title('📊 Anomaly Count by Severity', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)

plt.tight_layout()
plt.show()

print("\n🚦 Severity Levels:")
for level, count in severity_counts.items():
    print(f"  {level}: {count} аномалий")

## 8️⃣ Summary Report

In [ ]:
print("="*60)
print("📋 EV-QA-FRAMEWORK ANALYSIS REPORT")
print("="*60)
print(f"\n📊 Dataset Statistics:")
print(f"  Total Samples: {len(results)}")
print(f"  Normal Samples: {len(results[~results['is_anomaly']])}")
print(f"  Anomalies Detected: {len(detected_anomalies)}")
print(f"  Anomaly Rate: {len(detected_anomalies)/len(results)*100:.2f}%")

print(f"\n🎯 Detection Performance:")
print(f"  True Anomalies Injected: {n_anomalies}")
print(f"  Detected by ML: {len(detected_anomalies)}")
print(f"  Detection Rate: {len(detected_anomalies)/n_anomalies*100:.1f}%")

print(f"\n🚨 Severity Breakdown:")
for level in ['CRITICAL', 'WARNING', 'INFO']:
    count = severity_counts.get(level, 0)
    print(f"  {level}: {count} ({count/len(detected_anomalies)*100:.1f}% of anomalies)")

print(f"\n⚡ Voltage Statistics:")
print(f"  Normal Range: {results[~results['is_anomaly']]['voltage'].min():.1f}V - {results[~results['is_anomaly']]['voltage'].max():.1f}V")
print(f"  Anomaly Range: {results[results['is_anomaly']]['voltage'].min():.1f}V - {results[results['is_anomaly']]['voltage'].max():.1f}V")

print(f"\n🌡️ Temperature Statistics:")
print(f"  Normal Range: {results[~results['is_anomaly']]['temp'].min():.1f}°C - {results[~results['is_anomaly']]['temp'].max():.1f}°C")
print(f"  Anomaly Range: {results[results['is_anomaly']]['temp'].min():.1f}°C - {results[results['is_anomaly']]['temp'].max():.1f}°C")

print("\n" + "="*60)
print("✅ Analysis Complete!")
print("="*60)

## 🎯 Выводы

**EV-QA-Framework успешно детектирует аномалии в телеметрии батареи:**

1. ✅ **Pydantic валидация** отсекает невалидные данные на входе
2. ✅ **Isolation Forest** находит аномалии в многомерном пространстве
3. ✅ **Severity classification** приоритизирует критичные проблемы
4. ✅ **Визуализация** помогает понять паттерны аномалий

**Применение в реальном мире:**
- Раннее обнаружение перегрева батареи → предотвращение thermal runaway
- Детекция аномальных скачков напряжения → защита BMS
- Снижение warranty costs через predictive maintenance

---

**GitHub:** https://github.com/remontsuri/EV-QA-Framework  
**License:** MIT (Free for commercial use)  
**Author:** @remontsuri